# __Exploratory Data Analysis (EDA)__
_Projet Analyse de sentiments — Exploration initiale des tweets_


In [1]:
import sys
import os

# On ajoute le chemin absolu vers src/
sys.path.append(os.path.abspath(".."))

import mlflow
from src.utils.tracking import log_run


# On fixe l'emplacement centralisé des logs MLflow (remonte d'un cran car le notebook est dans /notebooks/)
mlflow.set_tracking_uri("file:../mlruns")

# On déclare (ou crée si nécessaire) l'expérience
mlflow.set_experiment("Projet7-Sentiment")

# Exemple de paramètres et métriques fictifs
params = {"C": 1.0, "penalty": "l2"}
metrics = {"accuracy": 0.82, "f1": 0.79}

# Lancement d’un premier run de test
log_run("Test-LogisticRegression", params, metrics)



In [2]:
import pandas as pd

# Documentation (commentaire) :
"""
CSV d’origine : training.1600000.processed.noemoticon.csv
Colonnes attendues (historiquement) :
  0: sentiment (0 = négatif, 4 = positif)
  1: id du tweet
  2: date/heure
  3: query (souvent "NO_QUERY")
  4: username
  5: text

Pipeline vers parquet (exemple générique) :
- lecture CSV avec encoding latin-1
- sélection colonnes [0,5] → renommage en ["label","text"]
- filtrage éventuel des lignes vides, normalisation minimale
- sauvegarde en parquet dans data/processed/tweets.parquet
"""


SOURCE_RAW = "../data/raw/training.1600000.processed.noemoticon.csv"  # fichier brut (non versionné)
SOURCE_PROCESSED = "../data/processed/tweets.parquet"                 # dérivé utilisé pour l'EDA

print("Données brutes (non chargées ici) :", SOURCE_RAW)
print("Données traitées (chargées)       :", SOURCE_PROCESSED)


Données brutes (non chargées ici) : ../data/raw/training.1600000.processed.noemoticon.csv
Données traitées (chargées)       : ../data/processed/tweets.parquet


In [3]:
# Charger le fichier parquet optimisé
df = pd.read_parquet("../data/processed/tweets.parquet")

# Aperçu des données
df.head()


,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


## __Dimensions du dataset__


In [6]:
df.shape


(1600000, 2)

## __Répartition des labels__


In [5]:
df['label'].value_counts(normalize=True)


label
0    0.5
4    0.5
Name: proportion, dtype: float64

## __Statistiques sur la longueur des tweets__


In [7]:
df['len'] = df['text'].str.len()
df['len'].describe()


count    1.600000e+06
mean     7.409011e+01
std      3.644114e+01
min      6.000000e+00
25%      4.400000e+01
50%      6.900000e+01
75%      1.040000e+02
max      3.740000e+02
Name: len, dtype: float64

In [4]:
# Aperçu types et valeurs manquantes
display(df.dtypes)
display(df.isna().sum().sort_values(ascending=False))

# Vérification du domaine de labels
labels_uniques = sorted(df["label"].unique().tolist())
print("Labels uniques :", labels_uniques)
assert set(labels_uniques).issubset({0,4}), "Les labels ne sont pas uniquement {0,4}."

# Doublons (au cas où)
nb_dupl_text = df["text"].duplicated().sum()
print("Doublons sur 'text' :", nb_dupl_text)


label     int64
text     object
dtype: object

label    0
text     0
dtype: int64

Labels uniques : [0, 4]
Doublons sur 'text' : 18534


## __Création d'un dictionnaire de données__

Un dictionnaire de données (ou data dictionary) est un document qui décrit chaque colonne du dataset.  
Il répond à trois objectifs :
- __Compréhension__ : savoir exactement ce que contient chaque champ (utile pour le créateur, mais aussi pour le lecteur).
- __Communication__ : si quelqu’un d’autre reprend ton projet, il/elle peut comprendre la structure des données sans replonger dans le CSV.
- __Traçabilité__ : noter les décisions (par ex. « colonne X supprimée car redondante »).

In [8]:
# Dictionnaire
dico = [
    {"colonne": "label", "type": str(df["label"].dtype), "description": "Sentiment binaire (0 = négatif, 4 = positif)", "exemple": df["label"].iloc[0]},
    {"colonne": "text",  "type": str(df["text"].dtype),  "description": "Contenu textuel du tweet (anglais)",          "exemple": df["text"].iloc[0][:80] + ("..." if len(df['text'].iloc[0])>80 else "")},
]

# Si 'len' pour l’EDA :
if "len" in df.columns:
    dico.append({"colonne": "len", "type": str(df["len"].dtype), "description": "Longueur (nb de caractères) calculée", "exemple": df["len"].iloc[0]})

data_dict = pd.DataFrame(dico)
display(data_dict)

# Exports (garde une trace lisible côté repo)
data_dict_path_csv = "../reports/data_dictionary.csv"
data_dict_path_md  = "../reports/data_dictionary.md"

data_dict.to_csv(data_dict_path_csv, index=False)

with open(data_dict_path_md, "w", encoding="utf-8") as f:
    f.write("# Data Dictionary\n\n")
    for _, r in data_dict.iterrows():
        f.write(f"## {r['colonne']}\n")
        f.write(f"- Type : `{r['type']}`\n")
        f.write(f"- Description : {r['description']}\n")
        f.write(f"- Exemple : {r['exemple']}\n\n")

print("Exporté :", data_dict_path_csv, "et", data_dict_path_md)


,colonne,type,description,exemple
0,label,int64,"Sentiment binaire (0 = négatif, 4 = positif)",0
1,text,object,Contenu textuel du tweet (anglais),"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
2,len,int64,Longueur (nb de caractères) calculée,115


Exporté : ../reports/data_dictionary.csv et ../reports/data_dictionary.md
